The GAP Species range data originates from a back-end database maintained by staff at the Coop Unit at North Carolina State University, where they have been used in the production of the GAP Species Habitat Distribution Models/Maps and for other purposes. The ranges are based on a derivation of the 12 digit hydrologic unit codes and watershed boundaries documented in [this item](https://www.sciencebase.gov/catalog/item/56d496eee4b015c306f17a42). The data from that backend database were exported to text files and make their full online "debut" under [this ScienceBase item](https://www.sciencebase.gov/catalog/item/5951527de4b062508e3b1e79) (item may still be private when you read this).

In order to get the data online for reference behind the habitat distribution models and for broader use in analyses, we loaded them up to the GC2 infrastructure we are using as part of the Biogeographic Information System work. Because the files are large enough, we opted for a bulk upload method that used a behind the scenes user interface to get the data into the PostgreSQL system of GC2 as opposed to iterating over each record in code via the API. That breaks the provenance chain a little bit and resulted in some post-upload cleanup. This notebook takes care of the cleanup and runs some tests to verify that all of the data are online and functioning the way they should be.

In [8]:
import requests
from bis2 import gc2
import pysb
from IPython.display import display

_baseURL = gc2.sqlAPI("DataDistillery","BCB")

In [3]:
sb = pysb.SbSession()
username = input("Username: ")
sb.loginc(str(username))

Username: sbristol@usgs.gov
········


### Download range data tables for checks

This block gets the range data item from ScienceBase and downloads the files for local processing, supporting the next section where we check the total number of records in each range data file. I did this initially using pysb because the item was restricted to authorized users. Once the item is public, this check could be replaced with code that simply reads the four individual files into memory for a quick check of the total records.

In [6]:
_sbRangeItemID = "5951527de4b062508e3b1e79"
_sbRangeItem = sb.get_item(_sbRangeItemID)
display (_sbRangeItem)
sb.get_item_files(_sbRangeItem)

{'body': 'GAP species range data are coarse representations of the total areal extent a species occupies, in other words the geographic limits within which a species can be found (Morrison and Hall 2002). These data provide the geographic extent within which the National Gap Analysis Program delineates areas of suitable habitat for terrestrial vertebrate species in their species habitat maps. The range maps are created by attributing a vector file derived from the 12-digit Hydrologic Unit Dataset (USDA NRCS 2009). Modifications to that dataset are described here (Provide link to the metadata record for the HUC dataset). Attribution of the season range for each species was based on the literature and online sources (See Table 1). Attribution for each hydrologic unit within the range included values for origin (native, introduced, reintroduced, vagrant), occurrence (extant, possibly present, potentially present, extirpated), reproductive use (breeding, non-breeding, both) and season (yea

downloading https://www.sciencebase.gov/catalog/file/get/5951527de4b062508e3b1e79?f=__disk__ca%2Fb2%2Ff5%2Fcab2f5b6701d5e57483c991e5a920aa3e6a10d9a to ./U_S_Geological_Survey_Gap_Analysis_Program_Species_Ranges5July2017.xml
downloading https://www.sciencebase.gov/catalog/file/get/5951527de4b062508e3b1e79?f=__disk__17%2Fef%2Fae%2F17efaeebae98d415b0bed5ca295e25890905d078 to ./HawaiiGAP.zip
downloading https://www.sciencebase.gov/catalog/file/get/5951527de4b062508e3b1e79?f=__disk__7a%2F88%2Fec%2F7a88ecfaf8653c03b97084c897eac7f665d83f9e to ./National_GAP_Amphibians_Range_Table.txt
downloading https://www.sciencebase.gov/catalog/file/get/5951527de4b062508e3b1e79?f=__disk__79%2F59%2Fbf%2F7959bf07fb7aac0af8007682ebee624ec67ff8fb to ./National_GAP_Reptiles_Range_Table.txt
downloading https://www.sciencebase.gov/catalog/file/get/5951527de4b062508e3b1e79?f=__disk__29%2Fa5%2Fab%2F29a5ab9af22fded74fa3973199b1f44cf729bf92 to ./National_GAP_Mammals_Range_Table.txt
downloading https://www.sciencebase

{'name': 'National_GAP_Birds_Range_Table.txt',
 'size': 612019794,
 'url': 'https://www.sciencebase.gov/catalog/file/get/5951527de4b062508e3b1e79?f=__disk__38%2F38%2Fdf%2F3838df4834d4cfb30bb63fef905a4a5aa678934d'}

### Check total records

This code checks the previously downloaded files against the resulting database tables to make sure that we got all the rows.

In [12]:
_tableNames = []
_tableNames.append("National_GAP_Birds_Range_Table")
_tableNames.append("National_GAP_Mammals_Range_Table")
_tableNames.append("National_GAP_Amphibians_Range_Table")
_tableNames.append("National_GAP_Reptiles_Range_Table")

for table in _tableNames:
    dbCheck = requests.get(_baseURL+"&q=SELECT count(*) AS num FROM gap."+table.lower()).json()
    dbCount = int(dbCheck["features"][0]["properties"]["num"])
    fileCount = sum(1 for line in open(table+".txt"))-1
    print (table, "In file:", fileCount, "In database:", dbCount)

National_GAP_Birds_Range_Table In file: 18546052 In database: 18546052
National_GAP_Mammals_Range_Table In file: 4882718 In database: 4882718
National_GAP_Amphibians_Range_Table In file: 1298264 In database: 1298264
National_GAP_Reptiles_Range_Table In file: 2272251 In database: 2272251


### Merge original tables

This code merges the four original tables uploaded from text files into a single table for all range data.

Note: When I first ran this I attempted to simply pull all of the data from the four tables directly into a new table with a CREATE TABLE AS statement. However, that caused an out of memory problem in the GC2 infrastructure, and I had to break things up. I think we resolved the infrastructure problem and might be able to run that query now to make this a little simpler, but the following two code blocks create the table and then iterate over the original source tables to insert all their records.

In [21]:
_qCreateTable = "CREATE TABLE IF NOT EXISTS gap.gapspeciesrange ( \
    sourcetable VARCHAR, \
    struc CHAR(6), \
    strhuc12rng CHAR(12), \
    intgaporigin INT, \
    intgappres INT, \
    intgaprepro INT, \
    intgapseas INT)"
print (requests.get(_baseURL+"&q="+_qCreateTable).json())


{'_execution_time': 0.027, 'affected_rows': 0, 'success': True}


In [22]:
for table in _tableNames:
    qInsert = "INSERT INTO gap.gapspeciesrange (SELECT '"+table+"' AS sourcetable, struc, strhuc12rng::text, intgaporigin, intgappres, intgaprepro, intgapseas FROM gap."+table.lower()+")"
    print (requests.get(_baseURL+"&q="+qInsert).json())

{'_execution_time': 95.024, 'affected_rows': 18546052, 'success': True, 'auth_check': {'auth_level': None, 'success': True, 'session': None}}
{'_execution_time': 33.612, 'affected_rows': 4882718, 'success': True, 'auth_check': {'auth_level': None, 'success': True, 'session': None}}
{'_execution_time': 2.767, 'affected_rows': 1298264, 'success': True, 'auth_check': {'auth_level': None, 'success': True, 'session': None}}
{'_execution_time': 7.648, 'affected_rows': 2272251, 'success': True, 'auth_check': {'auth_level': None, 'success': True, 'session': None}}


### Fix HUC code problem

The simple upload process we used on the text files decided to put the 12 digit codes (originally HUC codes) in as numeric values instead of strings. We corrected for that when we cast those integers to text and inserted them into the new merged table as strings. However, we need to fix a problem with leading 0s getting lopped off.

In [37]:
_qCheckCodes = "UPDATE gap.gapspeciesrange SET strhuc12rng = '0' || strhuc12rng WHERE length(strhuc12rng) < 12"
print (requests.get(_baseURL+"&q="+_qCheckCodes).json())

{'_execution_time': 169.255, 'affected_rows': 10715392, 'success': True, 'auth_check': {'auth_level': 'Write', 'success': True, 'session': None, 'checked_relations': ['gap.gapspeciesrange']}}


In [38]:
# This last query just checks to make sure there aren't any other unexpected strange values in the data. All huc codes should be 12 characters long.
print (requests.get(_baseURL+"&q=SELECT count(*) AS num FROM gap.gapspeciesrange WHERE length(strhuc12rng) <> 12").json())

{'features': [{'type': 'Feature', 'properties': {'num': 0}}], '_execution_time': 46.487, 'success': True, 'forStore': [{'name': 'num', 'type': 'bigint'}], 'type': 'FeatureCollection', 'auth_check': {'auth_level': 'Write', 'success': True, 'session': None, 'checked_relations': ['gap.gapspeciesrange']}, 'forGrid': [{'dataIndex': 'num', 'type': 'bigint', 'typeObj': None, 'header': 'num'}]}
